This notebook contains quick throwaway code for MLAB w3d4.

### Load dependencies

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch as t
import transformers

### Load gpt2

In [3]:
gpt2 = transformers.GPT2LMHeadModel.from_pretrained('gpt2').cuda()
# ref_gpt2 = transformers.GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = transformers.GPT2TokenizerFast.from_pretrained('gpt2')

In [4]:
prompt_tensor = t.tensor(
    [gpt2.config.eos_token_id],
    dtype=t.long,
    device=gpt2.device,
)

batch_size: int = 80
gen_len: int = 20
temp: float = 0.6

gen_tensor = gpt2.generate(
    input_ids=prompt_tensor.reshape(1, -1),
    min_length=len(prompt_tensor) + gen_len,
    max_length=len(prompt_tensor) + gen_len,
    do_sample=True,
    temperature=temp,
    top_k=gpt2.config.vocab_size,
    top_p=1.0,
    num_return_sequences=batch_size,
    use_cache=True,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [5]:
print(gen_tensor.shape)
gen_texts: list[str] = tokenizer.batch_decode(gen_tensor[:, 1:])
gen_texts[:2]

torch.Size([80, 21])


['\nThis article is about the player in the original Dragon Ball Dragon Ball Super series. For the character',
 '\nThe United States has been in a state of war with Russia over Ukraine since 2014.\n\n']

In [6]:
def count_periods(s: str) -> int:
    return s.count(".")

rewards = t.tensor(
    [count_periods(t) for t in gen_texts],
    dtype=t.long,
    device=gpt2.device,
)
rewards

tensor([ 1,  1,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  1,  1,  0,  0,  1,  1,  2,  0,  0,  1,  0,  0,  0,  1,  1,  1,
         1, 20,  1,  0,  0,  1,  1,  0,  0,  0,  0,  1,  1,  1,  0,  0,  2,  0,
         0,  0,  1,  2,  0,  0,  1,  1,  1,  0,  0,  0,  0,  1,  1,  0,  1,  0,
         0,  0,  1,  1,  0,  1,  0,  0], device='cuda:0')